In [5]:
import numpy as np

class DecisionTree:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth

    def fit(self, X, y):
        self.X = X
        self.y = y
        self.root = self._build_tree(X, y, depth=0)

    def _build_tree(self, X, y, depth):
        # print('begin _build_tree', X.shape, y.shape)
        n_samples, n_features = X.shape
        unique_classes, count = np.unique(y, return_counts=True)
        # print('unique_classes', unique_classes, 'counts:', counts)


        # end ealry
        if len(unique_classes) == 1 or  depth == self.max_depth:
            # print('bingo stop')
            return unique_classes[np.argmax(count)]
        
        # To split, split with each feature
        # with each feature, split by each value, save the one with best gini
        best_split = None
        best_gini = 1.0

        for feature_idx in range(n_features):
            
            feature = X[:, feature_idx]
            unique_values = np.unique(feature)
            # print('unique_values', len(unique_values), unique_values)

            for value in unique_values:
                left_mask = X[:, feature_idx] <= value # return mask True or False (can work as index)
                right_mask = X[:, feature_idx] > value

                if len(y[left_mask]) >= 1 and len(y[right_mask]) >= 1:
                    left_gini = self._gini(y[left_mask])
                    right_gini = self._gini(y[right_mask])
                    split_gini = len(y[left_mask]) / n_samples * left_gini + len(y[right_mask]) / n_samples * right_gini
                    
                    if split_gini < best_gini:
                        best_gini = split_gini
                        best_split = (feature_idx, value)

        # if the split didn't give a good result, just return class with the highest rate
        if best_gini == 1.0:
            return unique_classes[np.argmax(count)]
        
        feature_idx, value = best_split
        left_mask = X[:, feature_idx] <= value
        right_mask = X[:, feature_idx] > value

        left_subtree = self._build_tree(X[left_mask], y[left_mask], depth+1)
        right_subtree = self._build_tree(X[right_mask], y[right_mask], depth+1)

        return (feature_idx, value, left_subtree, right_subtree)



    def _gini(self, y):
        _, count = np.unique(y, return_counts=True)
        rate = count / len(y)
        gini = 1.0 - np.sum(rate ** 2)
        return gini

    def predict(self, X):
        predictions = []
        for x in X:
            # print('self.root', self.root)
            predict = self._predict(x, self.root)
            predictions.append(predict)
        return np.array(predictions)

    def _predict(self, x, node):
        # The base code: if isinstance(node, int32)
        # But the result can be int64 and int, we should do like below:
        if not isinstance(node, tuple):
            return node
        # print('node', node)
        feature_idx, value, left_subtree, right_subtree = node
        if x[feature_idx] <= value:
            return self._predict(x, left_subtree)
        else:
            return self._predict(x, right_subtree)



In [6]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

data = load_iris()
X = data.data
y = data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2,
                                                    random_state=42)

In [7]:
X.shape, y.shape

((150, 4), (150,))

In [8]:
tree = DecisionTree(max_depth=2)
tree.fit(X_train, y_train)
train_accuracy = accuracy_score(y_train, tree.predict(X_train))
test_accuracy = accuracy_score(y_test, tree.predict(X_test))

print(f"Train acuracy: {train_accuracy}")
print(f"Test accuracy: {test_accuracy}")

Train acuracy: 0.95
Test accuracy: 0.9666666666666667
